- issues https://github.com/onnx/tensorflow-onnx/issues/1260
-
- onnx official https://onnxruntime.ai/docs/tutorials/tf-get-started.html
-
- how convert universal encoder  https://github.com/microsoft/onnxruntime-extensions/blob/main/tutorials/tf2onnx_custom_ops_tutorial.ipynb

- pip install onnxruntime-extensions
- pip install tf2onnx
- pip install onnxruntime

- python 3.7+

!python -m tf2onnx.convert --saved-model /path/to/savemodel/folder --output /path/to/new_savemodel/folder/model1.onnx --extra_opset ai.onnx.contrib:1

 OR

!python -m tf2onnx.convert --saved-model /home/farid/F_Drive/resume_web_app/local_model --output /home/farid/F_Drive/resume_web_app/local_model/use_v5.onnx --opset 12 --extra_opset ai.onnx.contrib:1 --tag server


In [63]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
from onnxruntime_extensions import get_library_path
import onnx
import onnxruntime as rt
import os
import time


In [3]:
print(onnx.__version__)
print(rt.__version__)

1.12.0
1.12.1


In [4]:
embed =  tf.saved_model.load('/home/farid/F_Drive/resume_web_app/local_model')
!echo "ONNX Runtime Execution Providers: " && python -c "import onnxruntime as ort; print(ort.get_available_providers())"


2022-10-17 15:11:25.812885: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-17 15:11:25.910039: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-17 15:11:25.910719: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-17 15:11:25.911723: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

ONNX Runtime Execution Providers: 
['CPUExecutionProvider']


In [5]:

ex_text_str = (
    "MEMPHIS, Tenn. – Four days ago, Jon Rahm was " 
    "enduring the season’s worst weather conditions on Sunday at The "
    "Open on his way to a closing 75 at Royal Portrush, which "
    "considering the wind and the rain was a respectable showing."
)
# org_emb = embed([ex_text_str]).numpy()


opt = rt.SessionOptions()
opt.register_custom_ops_library(get_library_path())
sess = rt.InferenceSession(os.path.abspath(os.path.join("onnx_model","use2_v5.onnx")),opt)


print("Inputs:", [inp.name for inp in sess.get_inputs()])
print("Outputs:", [out.name for out in sess.get_outputs()])

Inputs: ['inputs']
Outputs: ['outputs']


## Execution time for ONNX

In [6]:
%%time
results_ort = sess.run(["outputs"], {"inputs": [ex_text_str]})[0][0]

CPU times: user 58 ms, sys: 0 ns, total: 58 ms
Wall time: 34.5 ms


## Execution time for TFHub model

In [25]:
%%time
results_tf = embed([ex_text_str]).numpy()[0]

CPU times: user 61.1 ms, sys: 4.99 ms, total: 66.1 ms
Wall time: 72.3 ms


In [8]:
for ort_res, tf_res in zip(results_ort, results_tf):
                                       
    np.allclose(ort_res, tf_res,  atol=1e-7)

print("Results match")


Results match


In [11]:
results_ort.shape

(512,)

In [12]:
results_tf.shape

(512,)

In [13]:
#Returns True if two arrays are element-wise equal within a tolerance.
np.allclose(results_ort, results_tf, atol=1e-7)

True

In [14]:
np.allclose( results_tf,results_ort, atol=1e-7)

True

In [17]:
results_ort

array([-1.69175193e-02, -3.90927866e-02,  6.93201646e-03,  4.84580882e-02,
        6.09382093e-02, -5.27378544e-03, -6.02557473e-02,  1.80724747e-02,
       -4.98561114e-02,  5.78437448e-02,  9.37279314e-03, -2.80513912e-02,
       -1.51188141e-02, -1.79427105e-03,  8.87294710e-02, -2.56456900e-02,
       -7.70685151e-02, -7.78886117e-03, -3.84930596e-02, -9.71681252e-02,
       -5.00230603e-02,  3.94539274e-02, -3.14658582e-02, -1.71176475e-02,
        5.43009862e-02, -3.27435024e-02, -3.38566527e-02,  4.40921225e-02,
       -8.14940259e-02, -4.06169854e-02, -1.79708079e-02, -9.18401696e-04,
       -6.50504827e-02,  3.70148011e-02,  6.79724813e-02,  6.38036355e-02,
       -3.32412645e-02,  5.52082397e-02, -3.15417722e-02, -1.42254550e-02,
        6.85458556e-02,  2.49837413e-02,  3.49380402e-03,  5.60050346e-02,
        2.54646735e-03, -3.07792202e-02,  1.86277479e-02,  2.17077658e-02,
        4.12676260e-02,  1.16574064e-01, -5.49769364e-02,  5.21534905e-02,
       -6.01469912e-02,  

In [18]:
results_tf

array([-1.69175118e-02, -3.90927419e-02,  6.93202717e-03,  4.84580621e-02,
        6.09382950e-02, -5.27379336e-03, -6.02557585e-02,  1.80724673e-02,
       -4.98560853e-02,  5.78436516e-02,  9.37277079e-03, -2.80514434e-02,
       -1.51188299e-02, -1.79428118e-03,  8.87294561e-02, -2.56457198e-02,
       -7.70685151e-02, -7.78888352e-03, -3.84930596e-02, -9.71681103e-02,
       -5.00230528e-02,  3.94539014e-02, -3.14658061e-02, -1.71176549e-02,
        5.43010533e-02, -3.27435061e-02, -3.38566303e-02,  4.40921485e-02,
       -8.14939812e-02, -4.06169407e-02, -1.79708246e-02, -9.18394129e-04,
       -6.50504827e-02,  3.70148234e-02,  6.79725036e-02,  6.38036802e-02,
       -3.32412533e-02,  5.52081615e-02, -3.15417275e-02, -1.42254410e-02,
        6.85459003e-02,  2.49837544e-02,  3.49376514e-03,  5.60049526e-02,
        2.54644477e-03, -3.07792388e-02,  1.86277460e-02,  2.17077490e-02,
        4.12676223e-02,  1.16574042e-01, -5.49769327e-02,  5.21535352e-02,
       -6.01469874e-02,  

In [22]:
a= ("hello how are you i am working on onnx to minimize the prediction time of model ok ")

## tfhub model execution time

In [23]:

%%time
h= embed([a])

CPU times: user 54.5 ms, sys: 12.8 ms, total: 67.3 ms
Wall time: 68.9 ms


## Onnx extecution time

In [24]:
%%time
s = sess.run(["outputs"], {"inputs": [a]})

CPU times: user 30.2 ms, sys: 202 µs, total: 30.4 ms
Wall time: 19 ms


In [36]:
print(h.shape)
h

(1, 512)


<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[ 1.75548308e-02, -2.53712125e-02,  6.53455183e-02,
         6.56028837e-03, -1.72246210e-02,  2.78889365e-03,
         6.56220317e-02, -1.64527670e-02, -2.09885109e-02,
        -5.22750765e-02, -2.35603028e-03, -6.60633435e-03,
        -3.67830358e-02,  3.12646083e-03,  4.60560732e-02,
         1.55256568e-02, -5.69774620e-02, -1.32739797e-01,
        -2.78589632e-02, -1.14863086e-03, -9.91843548e-03,
        -7.56925568e-02,  1.96667202e-02,  2.99145514e-03,
         5.27865663e-02,  3.85292880e-02, -5.11085764e-02,
        -3.31604965e-02,  2.24736631e-02,  1.09700412e-02,
         6.28691807e-04,  1.64564662e-02, -2.77873259e-02,
         2.52348837e-02, -3.67392413e-02,  5.03577814e-02,
         7.67236426e-02, -3.65411863e-02,  4.19895835e-02,
        -6.96063694e-03,  2.42230501e-02, -1.45296501e-02,
        -1.89536400e-02,  2.51740292e-02,  5.31111620e-02,
         3.39770839e-02, -5.05078770e-02, -5.17136827e-02,
      

In [34]:
print(s[0].shape)
s[0]

(1, 512)


array([[ 1.75547563e-02, -2.53711790e-02,  6.53455332e-02,
         6.56029489e-03, -1.72246434e-02,  2.78885756e-03,
         6.56220913e-02, -1.64527167e-02, -2.09885463e-02,
        -5.22751063e-02, -2.35598092e-03, -6.60632830e-03,
        -3.67830768e-02,  3.12643289e-03,  4.60560396e-02,
         1.55256446e-02, -5.69774285e-02, -1.32739812e-01,
        -2.78589204e-02, -1.14864623e-03, -9.91838425e-03,
        -7.56925717e-02,  1.96667071e-02,  2.99144280e-03,
         5.27865253e-02,  3.85292843e-02, -5.11086211e-02,
        -3.31605002e-02,  2.24736501e-02,  1.09700458e-02,
         6.28622482e-04,  1.64564475e-02, -2.77872402e-02,
         2.52348706e-02, -3.67391892e-02,  5.03577292e-02,
         7.67236650e-02, -3.65411714e-02,  4.19896245e-02,
        -6.96065836e-03,  2.42230464e-02, -1.45296874e-02,
        -1.89536382e-02,  2.51740534e-02,  5.31110801e-02,
         3.39770466e-02, -5.05078956e-02, -5.17136492e-02,
        -3.95798720e-02, -1.23752877e-01, -5.63319214e-0

In [64]:
lrr=("Contrary to popular belief, Lorem Ipsum is not simply random text. It has roots in a piece of classical Latin literature from 45 BC, making it over 2000 years old Richard McClintock a Latin professor at Hampden-Sydney College in Virginia, looked up one of the more obscure Latin words, consectetur, from a Lorem Ipsum passage, and going through the cites of the word in classical literature, discovered the undoubtable source. Lorem Ipsum comes from sections 1.10.32 and 1.10.33 of de Finibus Bonorum et MalorumThe Extremes of Good and Evil) by Cicero, written in 45 BC. This book is a treatise on the theory of ethics, very popular during the Renaissance. The first line of Lorem Ipsum, Lorem ipsum dolor sit amet.., comes from a line in section 1.10.32The standard chunk of Lorem Ipsum used since the 1500s is reproduced below for those interested. Sections 1.10.32 and 1.10.33 from de Finibus Bonorum et Malorum by Cicero are also reproduced in their exact original form, accompanied by English versions from the 1914 translation by H. Rackham.")

In [65]:

st = time.time()
tfhub= embed([lrr])
end = time.time()
print(f"Time taken: {(end-st)*10**3:.03f}ms")

Time taken: 119.186ms


In [71]:
st = time.time()
g= sess.run(["outputs"], {"inputs": [lrr]})
end = time.time()
print(f"Time taken: {(end-st)*10**3:.02f} ms")

Time taken: 142.18 ms


In [67]:
tfhub

<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[-4.51713167e-02,  3.51613462e-02, -5.32318912e-02,
         4.04510386e-02,  3.09496336e-02,  4.04979894e-03,
         3.28148529e-02, -1.06681220e-03, -1.08013265e-01,
         2.40424871e-02,  2.57976651e-02,  2.93743201e-02,
        -6.68065622e-02,  4.73733433e-02,  5.89743145e-02,
        -6.31595179e-02,  9.30976309e-03, -2.41598766e-02,
         3.15650851e-02, -1.45050939e-02, -1.48142185e-02,
         6.85573965e-02,  5.01207560e-02,  8.78098607e-02,
        -2.12710015e-02,  4.41263765e-02,  5.06177619e-02,
        -3.01392395e-02,  3.01248860e-02,  3.07259746e-02,
         4.87237563e-03, -4.45354395e-02,  6.57498604e-03,
        -3.67198465e-03, -3.28021571e-02,  8.80818698e-04,
        -1.01829309e-03,  1.14940880e-02,  3.63127738e-02,
         2.14055087e-02,  2.38539614e-02,  6.09835759e-02,
        -1.23464614e-02, -8.06562882e-03, -1.47221144e-02,
         1.33449659e-02, -2.84170602e-02, -9.91058648e-02,
      

In [68]:
g[0]

array([[-4.51713391e-02,  3.51613648e-02, -5.32318987e-02,
         4.04509678e-02,  3.09496485e-02,  4.04981058e-03,
         3.28148752e-02, -1.06680952e-03, -1.08013280e-01,
         2.40424797e-02,  2.57976353e-02,  2.93743722e-02,
        -6.68065473e-02,  4.73733507e-02,  5.89743443e-02,
        -6.31595328e-02,  9.30976123e-03, -2.41599046e-02,
         3.15651298e-02, -1.45050855e-02, -1.48142232e-02,
         6.85574338e-02,  5.01207449e-02,  8.78098682e-02,
        -2.12709531e-02,  4.41264100e-02,  5.06177917e-02,
        -3.01393047e-02,  3.01248673e-02,  3.07259690e-02,
         4.87235654e-03, -4.45354804e-02,  6.57495717e-03,
        -3.67196347e-03, -3.28020863e-02,  8.80776555e-04,
        -1.01826957e-03,  1.14940582e-02,  3.63127887e-02,
         2.14055404e-02,  2.38539986e-02,  6.09835722e-02,
        -1.23464242e-02, -8.06558505e-03, -1.47220939e-02,
         1.33449640e-02, -2.84170657e-02, -9.91058573e-02,
         9.16847377e-04,  5.64202201e-03, -1.60877462e-0